<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Preprocess Data
</div>

In [1]:
import pandas as pd
#check

### Read raw data that we have collected

In [2]:
raw_df = pd.read_csv("../data/raw/raw_data.csv")
raw_df.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,1961,ha,1000.0,E,Estimated value,NaN
1,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,1961,100 g/ha,7000.0,E,Estimated value,NaN
2,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,1961,t,700.0,E,Estimated value,NaN
3,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,1962,ha,1000.0,E,Estimated value,NaN
4,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,1962,100 g/ha,7000.0,E,Estimated value,NaN


-   Look like the `Note` column is all NaN, let check it:
    -   How many kind of data in the column?
    -   The ratio between data with no values and these other?

In [3]:
# check the variety of value
print("Unique values in the column:",raw_df['Note'].unique())

# check the ratio between loss and other data
raw_df['Note'].count()

Unique values in the column: [nan 'Unofficial figure']


548

-   Look like it's contain some useful information, so we keep it

### Meaning of each columns and rows

// Explain here

In [4]:
#TODO: Use code to explain if need

### Clear duplicates columns:
-   There is `Year Code` and `Year` columns, they have the same values, so we drop the `Year Code`.

In [5]:
raw_df = raw_df.drop(columns=['Year Code'])
raw_df.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year,Unit,Value,Flag,Flag Description,Note
0,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,ha,1000.0,E,Estimated value,NaN
1,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,100 g/ha,7000.0,E,Estimated value,NaN
2,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,t,700.0,E,Estimated value,NaN
3,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,ha,1000.0,E,Estimated value,NaN
4,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,100 g/ha,7000.0,E,Estimated value,NaN


### Check for duplicate rows

In [6]:
index = raw_df.index
detectDupSeries = index.duplicated(keep='first')
num_duplicated_rows = detectDupSeries.sum()

In [7]:
if num_duplicated_rows == 0:
    print(f"Your raw data have no duplicated line.!")
else:
    if num_duplicated_rows > 1:
        ext = "lines"
    else:
        ext = "line"
    print(f"Your raw data have {num_duplicated_rows} duplicated " + ext + ". Please de-deduplicate your raw data.!")

Your raw data have no duplicated line.!


### Check for the distribution of each value associated with an item.

In [8]:
def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def median(df):
    return (df.quantile(0.5)).round(1)

def lower_quartile(df):
    return (df.quantile(0.25)).round(1)

def upper_quartile(df):
    return (df.quantile(0.75)).round(1)

In [9]:
# make a DataFrame to store each item info about how their data distribute
data_distribute_df = pd.DataFrame()

for item in raw_df['Item'].unique():
    item_df = raw_df[raw_df['Item'] == item] # get rows that have the same item

    num_col_info_df = item_df.select_dtypes(exclude='object')
    num_col_info_df = num_col_info_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])

    num_col_info_df = num_col_info_df.transpose()

    num_col_info_df['Item'] = item
    
    data_distribute_df = pd.concat([data_distribute_df, num_col_info_df])

In [10]:
data_distribute_df

,missing_ratio,min,lower_quartile,median,upper_quartile,max,Item
Area Code,0.0,237.0,237.0,237.0,237.0,237.00,"Anise, badian, coriander, cumin, caraway, fenn..."
Element Code,0.0,5312.0,5312.0,5419.0,5510.0,5510.00,"Anise, badian, coriander, cumin, caraway, fenn..."
Item Code,0.0,711.0,711.0,711.0,711.0,711.00,"Anise, badian, coriander, cumin, caraway, fenn..."
Year,0.0,1961.0,1976.0,1991.0,2006.0,2021.00,"Anise, badian, coriander, cumin, caraway, fenn..."
Value,0.0,300.0,1000.0,4200.0,6885.5,25817.00,"Anise, badian, coriander, cumin, caraway, fenn..."
...,...,...,...,...,...,...,...
Area Code,0.0,237.0,237.0,237.0,237.0,237.00,Vegetables Primary
Element Code,0.0,5312.0,5312.0,5419.0,5510.0,5510.00,Vegetables Primary
Item Code,0.0,1735.0,1735.0,1735.0,1735.0,1735.00,Vegetables Primary
Year,0.0,1961.0,1976.0,1991.0,2006.0,2021.00,Vegetables Primary


### Determine the features that have a large number of missing values

In [11]:
percent_missing = raw_df.isnull().sum() * 100 / len(raw_df)
missing_value_df = pd.DataFrame({'column_name': raw_df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
Domain Code,Domain Code,0.000000
Domain,Domain,0.000000
Area Code,Area Code,0.000000
Area,Area,0.000000
Element Code,Element Code,0.000000
Element,Element,0.000000
Item Code,Item Code,0.000000
Item,Item,0.000000
Year,Year,0.000000
Unit,Unit,0.000000


-   We gonna choose >= 75% is missing a lot of values, so we have to drop the `Note` column (~96.364%).

In [12]:
raw_df = raw_df.drop(columns='Note')
raw_df

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year,Unit,Value,Flag,Flag Description
0,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,ha,1000.00,E,Estimated value
1,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,100 g/ha,7000.00,E,Estimated value
2,QCL,Crops and livestock products,237,Viet Nam,5510,Production,711,"Anise, badian, coriander, cumin, caraway, fenn...",1961,t,700.00,E,Estimated value
3,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,ha,1000.00,E,Estimated value
4,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,711,"Anise, badian, coriander, cumin, caraway, fenn...",1962,100 g/ha,7000.00,E,Estimated value
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15065,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,1735,Vegetables Primary,2020,100 g/ha,170584.00,E,Estimated value
15066,QCL,Crops and livestock products,237,Viet Nam,5510,Production,1735,Vegetables Primary,2020,t,17006433.11,E,Estimated value
15067,QCL,Crops and livestock products,237,Viet Nam,5312,Area harvested,1735,Vegetables Primary,2021,ha,1002974.00,E,Estimated value
15068,QCL,Crops and livestock products,237,Viet Nam,5419,Yield,1735,Vegetables Primary,2021,100 g/ha,171731.00,E,Estimated value


### Check data type of each columns

In [13]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15070 entries, 0 to 15069
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       15070 non-null  object 
 1   Domain            15070 non-null  object 
 2   Area Code         15070 non-null  int64  
 3   Area              15070 non-null  object 
 4   Element Code      15070 non-null  int64  
 5   Element           15070 non-null  object 
 6   Item Code         15070 non-null  int64  
 7   Item              15070 non-null  object 
 8   Year              15070 non-null  int64  
 9   Unit              15070 non-null  object 
 10  Value             15070 non-null  float64
 11  Flag              15070 non-null  object 
 12  Flag Description  15070 non-null  object 
dtypes: float64(1), int64(4), object(8)
memory usage: 1.5+ MB


`Area Code`, `Element Code`, `Item Code`, `Year` have numeric type. However, their magnitude does not have a significance. And, `Year` actually represents a period instead of a number, `Area Code`, `Element Code`, `Item Code` is just a category. Thus, they can be convert to categorical type.

In [14]:
for col in raw_df.drop(columns= 'Value').columns:
    raw_df[col] = raw_df[col].astype(str)
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15070 entries, 0 to 15069
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Domain Code       15070 non-null  object 
 1   Domain            15070 non-null  object 
 2   Area Code         15070 non-null  object 
 3   Area              15070 non-null  object 
 4   Element Code      15070 non-null  object 
 5   Element           15070 non-null  object 
 6   Item Code         15070 non-null  object 
 7   Item              15070 non-null  object 
 8   Year              15070 non-null  object 
 9   Unit              15070 non-null  object 
 10  Value             15070 non-null  float64
 11  Flag              15070 non-null  object 
 12  Flag Description  15070 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.5+ MB


### Observation about the distribution of data
-   

### Save processed data to a new csv file

In [15]:
raw_df.to_csv("../data/processed/processed_data.csv", index= False)